<a href="https://colab.research.google.com/github/yukinaga/hopfield_boltzmann/blob/main/section_3/02_boltzmann_operation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ボルツマンマシンの動作
BoltzmannMachineクラスを使い、ボルツマンマシンの動作を確認します。

## BoltzmannMachineクラス

In [ ]:
import numpy as np

class BoltzmannMachine:
    def __init__(self, num_neurons, temperature=1.0):
        """ボルツマンマシンの初期化"""
        self.num_neurons = num_neurons
        self.weights = np.random.randn(num_neurons, num_neurons) * 0.1
        np.fill_diagonal(self.weights, 0)  # 自己結合はゼロ
        self.biases = np.zeros(num_neurons)
        self.temperature = temperature  # 温度パラメータ

    def sigmoid(self, x):
        """シグモイド関数（温度パラメータを考慮）"""
        return 1 / (1 + np.exp(-x / self.temperature))

    def energy(self, state):
        """エネルギー関数"""
        return -0.5 * np.dot(state, np.dot(self.weights, state)) - np.dot(self.biases, state)

    def sample_neuron(self, state, idx):
        """単一のニューロンの状態を確率的に更新"""
        activation = np.dot(self.weights[idx], state) + self.biases[idx]
        prob = self.sigmoid(activation)
        return 1 if np.random.rand() < prob else 0

    def sample_state(self, state, steps=1):
        """全ニューロンの状態をランダムに更新"""
        for _ in range(steps):
            for i in range(self.num_neurons):
                state[i] = self.sample_neuron(state, i)
        return state

    def train(self, data, epochs=1000, learning_rate=0.01):
        """ボルツマンマシンの学習"""
        num_samples = data.shape[0]
        for epoch in range(epochs):
            for sample in data:
                positive_state = sample.copy()
                negative_state = sample.copy()

                # 負方向のフェーズ（ギブスサンプリング）
                for _ in range(10):
                    negative_state = self.sample_state(negative_state)

                # 重みとバイアスの更新
                for i in range(self.num_neurons):
                    for j in range(self.num_neurons):
                        self.weights[i, j] += learning_rate * (
                            positive_state[i] * positive_state[j] - negative_state[i] * negative_state[j]
                        )
                self.biases += learning_rate * (positive_state - negative_state)

            # 温度を徐々に下げる（アニーリング）
            self.temperature *= 0.99

            if (epoch + 1) % 100 == 0:
                print(f"Epoch {epoch + 1}/{epochs} completed, Temperature: {self.temperature:.4f}")

    def run(self, state, steps=10):
        """学習済みボルツマンマシンでデータを生成"""
        return self.sample_state(state, steps)


## BoltzmannMachineの利用
ボルツマンマシンを訓練し、パターンを再現します。

In [ ]:
import matplotlib.pyplot as plt

def plot_image(vector, title, shape=(8, 8)):
    """1次元配列を画像として表示"""
    plt.imshow(vector.reshape(shape), cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

# サンプルデータの作成（8x8の単純なパターン）
pattern1 = np.array([
    1, 1, 1, -1, -1, 1, 1, 1,
    1, -1, -1, -1, -1, -1, 1, 1,
    1, -1, 1, 1, 1, -1, 1, 1,
    1, -1, 1, -1, 1, -1, 1, 1,
    1, -1, 1, 1, 1, -1, 1, 1,
    1, -1, -1, -1, -1, -1, 1, 1,
    1, 1, 1, -1, -1, 1, 1, 1,
    1, 1, 1, 1, 1, 1, 1, 1
])

pattern2 = np.array([
    -1, -1, 1, 1, 1, 1, -1, -1,
    -1, 1, -1, -1, -1, -1, 1, -1,
    1, -1, -1, -1, -1, -1, -1, 1,
    1, -1, -1, -1, -1, -1, -1, 1,
    1, -1, -1, -1, -1, -1, -1, 1,
    1, -1, -1, -1, -1, -1, -1, 1,
    -1, 1, -1, -1, -1, -1, 1, -1,
    -1, -1, 1, 1, 1, 1, -1, -1
])

# データの準備
data = np.array([pattern1, pattern2])

# 元のパターンを画像として表示
plot_image(pattern1, "Original Pattern 1")
plot_image(pattern2, "Original Pattern 2")

# ボルツマンマシンの初期化と学習
bm = BoltzmannMachine(num_neurons=64, temperature=5.0)
bm.train(data, epochs=500, learning_rate=0.01)

# 学習結果を使ってパターンを再生成
generated_pattern1 = bm.run(pattern1)
plot_image(generated_pattern1, "Generated Pattern from Pattern 1")

generated_pattern2 = bm.run(pattern2)
plot_image(generated_pattern2, "Generated Pattern from Pattern 2")


以下、上記のコードを解説します。  

### **画像の表示関数**
```python
def plot_image(vector, title, shape=(8, 8)):
    """1次元配列を画像として表示"""
    plt.imshow(vector.reshape(shape), cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()
```
- 1次元配列を8×8の画像として表示します。

---

### **サンプルデータの作成**
```python
pattern1 = np.array([...])
pattern2 = np.array([...])
data = np.array([pattern1, pattern2])
```
- ここでは、2つの8×8のパターンをサンプルデータとして使用しています。

---

### **ボルツマンマシンの学習と生成**
```python
bm = BoltzmannMachine(num_neurons=64, temperature=5.0)
bm.train(data, epochs=500, learning_rate=0.01)

# 学習結果を使ってパターンを再生成
generated_pattern1 = bm.run(pattern1)
plot_image(generated_pattern1, "Generated Pattern from Pattern 1")

generated_pattern2 = bm.run(pattern2)
plot_image(generated_pattern2, "Generated Pattern from Pattern 2")
```
- ボルツマンマシンを初期化し、500エポックで学習させています。
- 学習したモデルを使って、与えられたパターンから新しいパターンを生成し、表示します。

---

### **まとめ**
ボルツマンマシンを用いて簡単なパターンの再生成を行いました。  
パターンを学習することで、ノイズの多い入力からも元のパターンを再構成できる能力を持つことが確認できました。